<small>
Part of the InnovatED STEM and DroneBlocks Land, Air, and Sea Robotics Curriculum  
Licensed for educational use in schools only.  
Redistribution, commercial use, or resale is strictly prohibited.  
© 2025 InnovatED STEM & DroneBlocks. All rights reserved.
</small>

# **LiDAR Functions Testing Notebook**
This notebook will allow you to test the **LiDAR sensor functions** using Python and ROS2.

### **Objectives**
- Start and stop the LiDAR sensor.
- Subscribe to the LiDAR data stream and analyze real-time readings.
- Detect obstacles using LiDAR distance data.
- Visualize the LiDAR scan using a 2D plot.

In [ ]:
# Import necessary libraries
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
import numpy as np
import matplotlib.pyplot as plt


## **1️⃣ Start LiDAR Sensor**
Run the following cell to start the **LiDAR launch file**.

In [ ]:
!ros2 launch your_package lidar.launch.py

## **2️⃣ Subscribe to LiDAR Data**
The following cell will create a **LiDAR reader node** and print distance values.

In [ ]:
class LidarReader(Node):
    def __init__(self):
        super().__init__('lidar_reader')
        self.subscription = self.create_subscription(
            LaserScan, '/scan_raw', self.lidar_callback, 10)
        self.subscription  # Prevent unused variable warning
    
    def lidar_callback(self, msg):
        print(f"Received LiDAR Data: {msg.ranges[:10]}")  # Print first 10 readings

rclpy.init()
lidar_node = LidarReader()
rclpy.spin_once(lidar_node)  # Process LiDAR data

## **3️⃣ Detect Obstacles**
If any obstacle is within **0.5 meters**, the function will display a warning.

In [ ]:
def check_obstacle(lidar_msg):
    min_distance = min(lidar_msg.ranges)
    if min_distance < 0.5:
        print("🚨 Obstacle detected within 0.5 meters!")
    else:
        print("✅ No obstacles detected within 0.5 meters.")

rclpy.spin_once(lidar_node)  # Process data
check_obstacle(lidar_node)

## **4️⃣ Visualize LiDAR Data**
This function will generate a **2D scatter plot** of the LiDAR readings.

In [ ]:
def plot_lidar(lidar_msg):
    angles = np.linspace(-np.pi, np.pi, len(lidar_msg.ranges))
    x = np.array(lidar_msg.ranges) * np.cos(angles)
    y = np.array(lidar_msg.ranges) * np.sin(angles)
    
    plt.scatter(x, y, s=1)
    plt.xlabel("X (meters)")
    plt.ylabel("Y (meters)")
    plt.title("LiDAR Scan")
    plt.show()

rclpy.spin_once(lidar_node)
plot_lidar(lidar_node)

## **5️⃣ Stop LiDAR Sensor**
Run this cell to **stop the LiDAR process** and free system resources.

In [ ]:
!ros2 service call /lidar_app/exit std_srvs/srv/Trigger

## **6️⃣ Shutdown Node**
Always **shutdown** the ROS2 node to prevent memory leaks.

In [ ]:
lidar_node.destroy_node()
rclpy.shutdown()
print('LiDAR Node Shutdown Complete.')